# 可转债二叉树定价模型演示

本 Notebook 使用本项目中的 **真实** 定价模块 `cb_arb.cb_pricing`，演示 CRR 二叉树构建、可转债定价与 Delta 计算。

所有数值结果均来自 `price_convertible_bond_binomial` 与 `build_stock_tree` 的实际调用，无虚构数据。

## 1. 设置路径并导入模块

确保能导入项目内的 `cb_arb` 包（在项目根目录或 `notebooks` 目录下运行均可）。

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd()
project_root = cwd if os.path.isdir(os.path.join(cwd, 'src')) else os.path.abspath(os.path.join(cwd, '..'))
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

from cb_arb.params import ConvertibleBondContract, TermStructure, CreditCurve
from cb_arb.cb_pricing import build_stock_tree, price_convertible_bond_binomial

# 导入 Notebook 工具函数（用于保存输出文件）
notebooks_dir = os.path.join(project_root, 'notebooks')
if notebooks_dir not in sys.path:
    sys.path.insert(0, notebooks_dir)
from notebook_utils import save_figure, get_figures_dir, get_data_dir

print("项目根目录:", project_root)
print("src 目录已添加到 Python 路径:", src_dir)
print("输出目录:", os.path.join(project_root, 'output'))
print("cb_arb 定价模块导入成功。")

## 2. CRR 股票价格树（真实 API）

使用 `build_stock_tree` 构建与 `cb_pricing` 内部一致的 CRR 树，用于理解定价输入。

In [ ]:
S0 = 100.0
maturity = 3.0
steps = 6
vol = 0.25
r_curve = TermStructure(rate_fn=lambda t: 0.02)
q_curve = TermStructure(rate_fn=lambda t: 0.01)

stock_tree, u, d, p, dt = build_stock_tree(
    S0=S0, maturity=maturity, steps=steps, vol=vol,
    r_curve=r_curve, q_curve=q_curve,
)

print("CRR 参数: u = %.4f, d = %.4f, p = %.4f, dt = %.4f" % (u, d, p, dt))
print("树形状:", stock_tree.shape)
print("第 0 步 (根):", stock_tree[0, 0])
print("第 1 步 (上/下):", stock_tree[1, 0], stock_tree[1, 1])
print("到期 (最后一行) 前几个节点:", stock_tree[-1, :4])

## 3. 可转债定价与 Delta（与 run_simple_backtest 一致参数）

使用与 `examples.run_simple_backtest` 中**完全相同**的合约与曲线参数，调用 `price_convertible_bond_binomial` 得到理论价与 Delta。

In [ ]:
contract = ConvertibleBondContract(
    face_value=100.0,
    coupon_rate=0.03,
    maturity=3.0,
    conversion_ratio=1.0,
    issue_price=100.0,
    call_price=None,
    put_price=None,
    coupon_freq=2,
)
r_curve = TermStructure(rate_fn=lambda t: 0.02)
q_curve = TermStructure(rate_fn=lambda t: 0.01)
credit_curve = CreditCurve(spread_fn=lambda t: 0.03)
steps = 50
vol = 0.25

price, delta = price_convertible_bond_binomial(
    S0=100.0,
    contract=contract,
    steps=steps,
    vol=vol,
    r_curve=r_curve,
    q_curve=q_curve,
    credit_curve=credit_curve,
)

print("S0 = 100.0 时:")
print("  可转债理论价格 (每张): %.4f" % price)
print("  Delta: %.4f" % delta)

## 4. 价格与 Delta 随股价变化（真实计算）

对一系列股价调用同一定价函数，得到真实的价格与 Delta 曲线。

In [ ]:
stock_prices = np.linspace(60.0, 140.0, 40)
prices = []
deltas = []
for S in stock_prices:
    p, d = price_convertible_bond_binomial(
        S0=S, contract=contract, steps=steps, vol=vol,
        r_curve=r_curve, q_curve=q_curve, credit_curve=credit_curve,
    )
    prices.append(p)
    deltas.append(d)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9, 7))
ax1.plot(stock_prices, prices, 'b-')
ax1.axhline(contract.face_value, color='gray', linestyle='--', label='面值')
ax1.set_xlabel('股价')
ax1.set_ylabel('可转债价格')
ax1.set_title('可转债理论价格 vs 股价 (cb_pricing 实际输出)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(stock_prices, deltas, 'g-')
ax2.axhline(contract.conversion_ratio, color='gray', linestyle='--', label='转换比例')
ax2.set_xlabel('股价')
ax2.set_ylabel('Delta')
ax2.set_title('Delta vs 股价 (cb_pricing 实际输出)')
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.tight_layout()

# 保存图片
save_figure(fig, '02_pricing_price_delta_vs_stock')

# 保存价格和 Delta 数据
from datetime import datetime
import pandas as pd
data_dir = get_data_dir()
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
price_delta_df = pd.DataFrame({
    'stock_price': stock_prices,
    'cb_price': prices,
    'delta': deltas
})
csv_path = data_dir / f'price_delta_vs_stock_{timestamp}.csv'
price_delta_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f'价格和 Delta 数据已保存到: {csv_path}')

plt.show()

## 5. 步数对价格与 Delta 的影响

固定其他参数，仅改变二叉树步数 `steps`，观察收敛情况（真实数值）。

In [ ]:
step_list = [10, 20, 30, 50, 80, 120, 200]
prices_by_steps = []
deltas_by_steps = []
for s in step_list:
    p, d = price_convertible_bond_binomial(
        S0=100.0, contract=contract, steps=s, vol=vol,
        r_curve=r_curve, q_curve=q_curve, credit_curve=credit_curve,
    )
    prices_by_steps.append(p)
    deltas_by_steps.append(d)

print("步数 -> (价格, Delta):")
for s, p, d in zip(step_list, prices_by_steps, deltas_by_steps):
    print("  %3d -> (%.4f, %.4f)" % (s, p, d))

plt.figure(figsize=(9, 4))
plt.subplot(1, 2, 1)
plt.plot(step_list, prices_by_steps, 'o-')
plt.xlabel('二叉树步数')
plt.ylabel('可转债价格')
plt.title('价格随步数变化')
plt.grid(True, alpha=0.3)
plt.subplot(1, 2, 2)
plt.plot(step_list, deltas_by_steps, 'o-')
plt.xlabel('二叉树步数')
plt.ylabel('Delta')
plt.title('Delta 随步数变化')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# 保存图片
save_figure(fig, '02_pricing_convergence_by_steps')

# 保存步数收敛数据
from datetime import datetime
import pandas as pd
data_dir = get_data_dir()
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
convergence_df = pd.DataFrame({
    'steps': step_list,
    'price': prices_by_steps,
    'delta': deltas_by_steps
})
csv_path = data_dir / f'pricing_convergence_{timestamp}.csv'
convergence_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f'步数收敛数据已保存到: {csv_path}')

plt.show()